In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns
import math

pd.options.mode.chained_assignment = None

## Crezione DataSet e analisi preliminari

In [2]:
dati =pd.read_excel('reportCompleto.xlsx')

In [3]:
dati.drop(columns=['Unnamed: 0'],axis=1,inplace=True)

In [4]:
colonne_vuote = list()

for column in dati.columns:
    if dati[column].isnull().all():
        colonne_vuote.append(column)

In [5]:
dati.drop(columns=colonne_vuote,axis=1,inplace=True)

In [6]:
dati.shape

(290, 2150)

In [7]:
index_horowitz = dati['record_id'].loc[dati['horowitz'].isnull()].index

In [8]:
index_horowitz

Int64Index([170, 185, 188, 192, 193, 206, 207, 253, 275], dtype='int64')

In [9]:
dati.drop(index_horowitz,inplace=True)
dati.reset_index(inplace=True)
dati.drop(columns=('index'),inplace=True)

In [10]:
index_eisto= dati[['esito_ti','dimissione','decesso']].loc[(dati['dimissione'].isnull()) & (dati['decesso'].isnull())].index

In [11]:
dati.drop(index_eisto,inplace=True)
dati.reset_index(inplace=True)
dati.drop(columns=('index'),inplace=True)

In [12]:
dati

,record_id,redcap_data_access_group,ricovero,tip_paz,tip_ricov_chirurg,tip_paz_chirurgico,tip_paz_chrurgico_altro,obiet_ricov_ti___1,obiet_ricov_ti___2,obiet_ricov_ti___3,...,dose_totale,ultrafiltr_totale,rac_cam_out,tip_camp_out___1,tip_camp_out___2,tip_camp_out___3,tip_camp_out___4,tip_camp_al_out,q_camp_out,outcome_ti_complete
0,4,NaN,2021-05-15,1,NaN,NaN,NaN,1,0,0,...,NaN,NaN,0.0,0,0,0,0,NaN,NaN,2
1,5,NaN,2021-02-07,1,NaN,NaN,NaN,1,0,0,...,NaN,NaN,NaN,0,0,0,0,NaN,NaN,2
2,6,NaN,2021-02-22,1,NaN,NaN,NaN,1,0,0,...,NaN,NaN,0.0,0,0,0,0,NaN,NaN,2
3,7,NaN,2020-12-30,1,NaN,NaN,NaN,1,0,0,...,NaN,NaN,NaN,0,0,0,0,NaN,NaN,2
4,8,NaN,2020-12-29,1,NaN,NaN,NaN,1,0,0,...,NaN,NaN,0.0,0,0,0,0,NaN,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,558-5,verona_azienda_osp,2022-10-23,1,NaN,NaN,NaN,0,1,0,...,NaN,NaN,0.0,0,0,0,0,NaN,NaN,2
273,558-8,verona_azienda_osp,2022-11-26,2,2.0,1.0,NaN,1,1,1,...,NaN,NaN,0.0,0,0,0,0,NaN,NaN,2
274,558-9,verona_azienda_osp,2022-11-28,1,NaN,NaN,NaN,1,1,1,...,NaN,NaN,0.0,0,0,0,0,NaN,NaN,2
275,558-10,verona_azienda_osp,2022-12-21,1,NaN,NaN,NaN,1,1,1,...,NaN,NaN,0.0,0,0,0,0,NaN,NaN,2


## Indicazioni al trattamento

In [13]:
ind_nefro= dati.columns.get_loc('nefrotox')
start_lattati= dati.columns.get_loc('unit_latt') 

In [14]:
dati.columns[ind_nefro:start_lattati]

Index(['nefrotox', 'fluids', 'emod', 'crea_ur', 'ipergl', 'mdc', 'flogosi',
       'indicaz_altro', 'colt', 'emp', 'atb_emp___1', 'atb_emp___2',
       'atb_emp___3', 'atb_emp___4', 'atb_emp___5', 'atb_emp___6',
       'atb_emp___7', 'atb_emp___8', 'atb_emp___9', 'atb_emp___10',
       'atb_emp___11', 'atb_emp___12', 'atb_emp___13', 'atb_emp___14',
       'atb_emp___15', 'atb_emp___16', 'atb_emp___17', 'atb_emp___18',
       'atb_emp___19', 'atb_emp___20', 'atb_emp___21', 'atb_emp___22',
       'atb_emp___23', 'atb_emp___24', 'atb_emp___25', 'atb_emp___26',
       'atb_emp___27', 'atb_emp___28', 'atb_emp___29', 'atb_emp___30',
       'atb_emp___31', 'atb_emp___32', 'atb_emp___33', 'atb_emp___34',
       'atb_emp___35', 'atb_emp___36', 'atb_emp___37', 'atb_emp___38',
       'atb_emp___39', 'atb_emp___40', 'atb_emp___41', 'atb_emp___42',
       'dose_ant', 'farm_dose'],
      dtype='object')

In [15]:
dati.drop(columns=list(dati.columns[ind_nefro:start_lattati]),inplace=True)

In [16]:
dati_sepsi = dati.loc[dati['target'] != 1 ]

In [17]:
dati_sepsi['lac_mmol'].fillna(dati_sepsi['lac_mg_mmol'],inplace=True)

In [18]:
dati['lac_mmol']=dati_sepsi['lac_mmol']

In [19]:
dati.drop(columns=['unit_latt', 'lac_mg', 'lac_mg_mmol','lac_mmol_mg','lac_2_4_mmol','lac_2_4_mg'],inplace=True)

In [20]:
dati.drop(columns=list(dati.columns)[dati.columns.get_loc('ipot'):dati.columns.get_loc('ipot')+5],inplace=True)

## Monitoraggi

In [21]:
first = dati.columns.get_loc('rass')
dati.drop(columns=list(dati.columns[first : dati.columns.get_loc('fc')]),inplace=True)


In [22]:
first_12h = dati.columns.get_loc('rass_foll_12h')
dati.drop(columns=list(dati.columns[first_12h : dati.columns.get_loc('fc_foll_12h')]),inplace=True)


In [23]:
first_24h= dati.columns.get_loc('rass_foll_24h')
dati.drop(columns=list(dati.columns[first_24h : dati.columns.get_loc('fc_foll_24h')]),inplace=True)


In [24]:
first2gg = dati.columns.get_loc('rass_foll_2gg')
dati.drop(columns=list(dati.columns[first2gg : dati.columns.get_loc('fc_foll_2gg')]),inplace=True)


In [25]:
first3gg = dati.columns.get_loc('rass_foll_3gg')
dati.drop(columns=list(dati.columns[first3gg : dati.columns.get_loc('fc_foll_3gg')]),inplace=True)


### Cuore

In [26]:
ind_adr=dati.columns.get_loc('adr_dos')
ind_altro = dati.columns.get_loc('dos_altro')
adr_12h = dati.columns.get_loc('adr_dos_foll_12h')
altro_12h = dati.columns.get_loc('dos_altro_foll_12h')
adr_24h = dati.columns.get_loc('adr_dos_foll_24h')
altro_24h = dati.columns.get_loc('dos_altro_foll_24h')
adr_2gg = dati.columns.get_loc('adr_dos_foll_2gg')
vis_2gg = dati.columns.get_loc('vis_foll_2gg')
adr_3gg = dati.columns.get_loc('adr_dos_foll_3gg')
altro_3gg = dati.columns.get_loc('dos_altro_foll_3gg')

In [27]:
dati[dati.columns[ind_adr:ind_altro]] = dati[dati.columns[ind_adr:ind_altro]].fillna(0)
dati[dati.columns[adr_12h:altro_12h]] = dati[dati.columns[adr_12h:altro_12h]].fillna(0)
dati[dati.columns[adr_24h:altro_24h]] = dati[dati.columns[adr_24h:altro_24h]].fillna(0)
dati[dati.columns[adr_2gg:vis_2gg]] = dati[dati.columns[adr_2gg:vis_2gg]].fillna(0)
dati[dati.columns[adr_3gg:altro_3gg]] = dati[dati.columns[adr_3gg:altro_3gg]].fillna(0)

In [28]:
dati.drop(columns=['dos_altro','dos_altro_foll_12h','dos_altro_foll_24h','dos_altro_foll_3gg'], inplace=True)

#### Monitoraggio cardiaco 

In [29]:
ind_monito= dati.columns.get_loc('mod_ecocardio')
ind_vent= dati.columns.get_loc('vent')
dati.drop(columns=list(dati.columns[ind_monito:ind_vent]),inplace=True)

In [30]:
monito_12h= dati.columns.get_loc('mod_ecocardio_foll_12h')
vent_12h= dati.columns.get_loc('vent_foll_12h')
dati.drop(columns=list(dati.columns[monito_12h:vent_12h]),inplace=True)

In [31]:
monito_24h= dati.columns.get_loc('mod_ecocardio_foll_24h')
vent_24h= dati.columns.get_loc('vent_foll_24h')
dati.drop(columns=list(dati.columns[monito_24h:vent_24h]),inplace=True)

In [32]:
monito_2gg= dati.columns.get_loc('mod_ecocardio_foll_2gg')
vent_2gg= dati.columns.get_loc('vent_foll_2gg')
dati.drop(columns=list(dati.columns[monito_2gg:vent_2gg]),inplace=True)


In [33]:
monito_3gg= dati.columns.get_loc('mod_ecocardio_foll_3gg')
vent_3gg= dati.columns.get_loc('vent_foll_3gg')
dati.drop(columns=list(dati.columns[monito_3gg:vent_3gg]),inplace=True)

### Polmoni


In [34]:
dati['hct'] = dati['hct'].fillna(30.0)
dati['hct_foll_12h'] = dati['hct_foll_12h'].fillna(30)
dati['hct_foll_24h'] = dati['hct_foll_24h'].fillna(30)
dati['hct_foll_2gg'] = dati['hct_foll_2gg'].fillna(30)
dati['hct_foll_3gg'] = dati['hct_foll_3gg'].fillna(30)

In [128]:
dati['fio2_foll_2gg'] = dati['fio2_foll_2gg'].fillna(dati['fio2_foll_3gg'])
dati['pao2_foll_2gg'] = dati['pao2_foll_2gg'].fillna(dati['pao2_foll_3gg'])
dati['horowitz_foll_2gg'] = dati['horowitz_foll_2gg'].fillna(dati['horowitz_foll_3gg'])

In [129]:
dati['fio2_foll_24h'] = dati['fio2_foll_24h'].fillna(dati['fio2_foll_2gg'])
dati['pao2_foll_24h'] = dati['pao2_foll_24h'].fillna(dati['pao2_foll_2gg'])
dati['horowitz_foll_24h'] = dati['horowitz_foll_24h'].fillna(dati['horowitz_foll_2gg'])

In [130]:
dati['fio2_foll_12h'] = dati['fio2_foll_12h'].fillna(dati['fio2_foll_24h'])
dati['pao2_foll_12h'] = dati['pao2_foll_12h'].fillna(dati['pao2_foll_24h'])
dati['horowitz_foll_12h'] = dati['horowitz_foll_12h'].fillna(dati['horowitz_foll_24h'])

In [131]:
dati['fio2'] = dati['fio2'].fillna(dati['fio2_foll_12h'])
dati['pao2'] = dati['pao2'].fillna(dati['pao2_foll_12h'])
dati['horowitz'] = dati['horowitz'].fillna(dati['horowitz_foll_12h'])

In [35]:
misura_lact = dati.columns.get_loc('misura_lact')
lact_mmol_mg_conver = dati.columns.get_loc('lact_mmol_mg_conver')
lattati_tempo0 = dati.iloc[:,misura_lact:lact_mmol_mg_conver+1]
lattati_tempo0=lattati_tempo0.fillna(0.0)
lattati_tempo0['lattati'] = lattati_tempo0['lact_mg_mmol_conver'] + lattati_tempo0['lact_mmol']
lattati_tempo0.replace(0.0,np.nan,inplace=True)
dati.insert(loc=misura_lact, column='lattati', value = lattati_tempo0['lattati'])
dati.drop(columns=['misura_lact','lact_mg','lact_mg_mmol_conver','lact_mmol','lact_mmol_mg_conver'],inplace=True)



In [36]:
misura_lact_12h = dati.columns.get_loc('misura_lact_foll_12h')
lact_mmol_mg_conver_12h = dati.columns.get_loc('lact_mmol_mg_conver_foll_12h')
lattati_tempo0_12h = dati.iloc[:,misura_lact_12h:lact_mmol_mg_conver_12h+1]
lattati_tempo0_12h=lattati_tempo0_12h.fillna(0.0)
lattati_tempo0_12h['lattati_foll_12h'] = lattati_tempo0_12h['lact_mg_mmol_conver_foll_12h'] + lattati_tempo0_12h['lact_mmol_foll_12h']
lattati_tempo0_12h.replace(0.0,np.nan,inplace=True)
dati.insert(loc=misura_lact_12h, column='lattati_foll_12h', value = lattati_tempo0_12h['lattati_foll_12h'])
dati.drop(columns=['misura_lact_foll_12h','lact_mg_foll_12h','lact_mg_mmol_conver_foll_12h','lact_mmol_foll_12h','lact_mmol_mg_conver_foll_12h'],inplace=True)



In [37]:
misura_lact_24h = dati.columns.get_loc('misura_lact_foll_24h')
lact_mmol_mg_conver_24h = dati.columns.get_loc('lact_mmol_mg_conver_foll_24h')
lattati_tempo0_24h = dati.iloc[:,misura_lact_24h:lact_mmol_mg_conver_24h+1]
lattati_tempo0_24h=lattati_tempo0_24h.fillna(0.0)
lattati_tempo0_24h['lattati_foll_24h'] = lattati_tempo0_24h['lact_mg_mmol_conver_foll_24h'] + lattati_tempo0_24h['lact_mmol_foll_24h']
lattati_tempo0_24h.replace(0.0,np.nan,inplace=True)
dati.insert(loc=misura_lact_24h, column='lattati_foll_24h', value = lattati_tempo0_24h['lattati_foll_24h'])
dati.drop(columns=['misura_lact_foll_24h','lact_mg_foll_24h','lact_mg_mmol_conver_foll_24h','lact_mmol_foll_24h','lact_mmol_mg_conver_foll_24h'],inplace=True)



In [38]:
misura_lact_2gg = dati.columns.get_loc('misura_lact_foll_2gg')
lact_mmol_mg_conver_2gg = dati.columns.get_loc('lact_mmol_mg_conver_foll_2gg')
lattati_tempo0_2gg = dati.iloc[:,misura_lact_2gg:lact_mmol_mg_conver_2gg+1]
lattati_tempo0_2gg=lattati_tempo0_2gg.fillna(0.0)
lattati_tempo0_2gg['lattati_foll_2gg'] = lattati_tempo0_2gg['lact_mg_mmol_conver_foll_2gg'] + lattati_tempo0_2gg['lact_mmol_foll_2gg']
lattati_tempo0_2gg.replace(0.0,np.nan,inplace=True)
dati.insert(loc=misura_lact_2gg, column='lattati_foll_2gg', value = lattati_tempo0_2gg['lattati_foll_2gg'])
dati.drop(columns=['misura_lact_foll_2gg','lact_mg_foll_2gg','lact_mg_mmol_conver_foll_2gg','lact_mmol_foll_2gg','lact_mmol_mg_conver_foll_2gg'],inplace=True)


In [39]:
misura_lact_3gg = dati.columns.get_loc('misura_lact_foll_3gg')
lact_mmol_mg_conver_3gg = dati.columns.get_loc('lact_mmol_mg_conver_foll_3gg')
lattati_tempo0_3gg = dati.iloc[:,misura_lact_3gg:lact_mmol_mg_conver_3gg+1]
lattati_tempo0_3gg=lattati_tempo0_3gg.fillna(0.0)
lattati_tempo0_3gg['lattati_foll_3gg'] = lattati_tempo0_3gg['lact_mg_mmol_conver_foll_3gg'] + lattati_tempo0_3gg['lact_mmol_foll_3gg']
lattati_tempo0_3gg.replace(0.0,np.nan,inplace=True)
dati.insert(loc=misura_lact_3gg, column='lattati_foll_3gg', value = lattati_tempo0_3gg['lattati_foll_3gg'])
dati.drop(columns=['misura_lact_foll_3gg','lact_mg_foll_3gg','lact_mg_mmol_conver_foll_3gg','lact_mmol_foll_3gg','lact_mmol_mg_conver_foll_3gg'],inplace=True)

In [40]:
dati['bicarb'] = dati['bicarb'].fillna(dati['be'] + 24.0 )
dati['bicarb'] = dati['bicarb'].fillna(24.0)
dati['be'] = dati['be'].fillna(dati['bicarb'] - 24.0)


dati['bicarb_foll_12h'] = dati['bicarb_foll_12h'].fillna(dati['be_foll_12h'] + 24.0 )
dati['bicarb_foll_12h'] = dati['bicarb_foll_12h'].fillna(24.0 )
dati['be_foll_12h'] = dati['be_foll_12h'].fillna(dati['bicarb_foll_12h'] - 24.0)


dati['bicarb_foll_24h'] = dati['bicarb_foll_24h'].fillna(dati['be_foll_24h'] + 24.0 )
dati['bicarb_foll_24h'] = dati['bicarb_foll_24h'].fillna(24.0 )
dati['be_foll_24h'] = dati['be_foll_24h'].fillna(dati['bicarb_foll_24h'] - 24.0)

dati['bicarb_foll_2gg'] = dati['bicarb_foll_2gg'].fillna(dati['be_foll_2gg'] + 24.0 )
dati['bicarb_foll_2gg'] = dati['bicarb_foll_2gg'].fillna(24.0 )
dati['be_foll_2gg'] = dati['be_foll_2gg'].fillna(dati['bicarb_foll_2gg'] - 24.0)

dati['bicarb_foll_3gg'] = dati['bicarb_foll_3gg'].fillna(dati['be_foll_3gg'] + 24.0 )
dati['bicarb_foll_3gg'] = dati['bicarb_foll_3gg'].fillna(24.0 )
dati['be_foll_3gg'] = dati['be_foll_3gg'].fillna(dati['bicarb_foll_3gg'] - 24.0)

### Reni

In [41]:
dati['creat_7_365']=dati['creat_7_365'].fillna(0.0)

In [42]:
dati.drop(columns=['diur', 'dos', 'diur_2', 'dos_2','dos_3'],inplace = True)
dati.drop(columns=['diur_foll_12h', 'dos_foll_12h', 'diur_2_foll_12h'],inplace = True)
dati.drop(columns=['diur_foll_24h', 'dos_foll_24h'],inplace = True)
dati.drop(columns=['diur_foll_2gg', 'dos_foll_2gg', 'diur_2_foll_2gg', 'dos_2_foll_2gg'],inplace = True)
dati.drop(columns=['diur_foll_3gg', 'dos_foll_3gg', 'diur_2_foll_3gg', 'dos_2_foll_3gg'],inplace = True)

In [43]:
dati.drop(columns=('nc'),inplace=True)

### Fegato

In [44]:
dati['bil'] = dati['bil'].fillna(dati['bil_foll_12h'])
dati['bil'] = dati['bil'].fillna(dati['bil_foll_24h'])
dati['bil'] = dati['bil'].fillna(dati['bil_foll_2gg'])


In [45]:
dati['bil']=dati['bil'].fillna(0.8)
dati['bil_foll_12h']=dati['bil_foll_12h'].fillna(0.8)
dati['bil_foll_24h']=dati['bil_foll_24h'].fillna(0.8)
dati['bil_foll_2gg']=dati['bil_foll_2gg'].fillna(0.8)
dati['bil_foll_3gg']=dati['bil_foll_3gg'].fillna(0.8)

In [46]:
indice_limon = dati.columns.get_loc('limon')
indice_teg = dati.columns.get_loc('ci_teg')+1
dati.drop(columns=dati.columns[indice_limon:indice_teg],inplace=True)


In [47]:
indice_limon_12h = dati.columns.get_loc('limon_foll_12h')
indice_teg_12h = dati.columns.get_loc('teg_foll_12h')+1
dati.drop(columns=dati.columns[indice_limon_12h:indice_teg_12h],inplace=True)


In [48]:
indice_limon_24h = dati.columns.get_loc('limon_foll_24h')
indice_teg_24h = dati.columns.get_loc('teg_foll_24h')+1
dati.drop(columns=dati.columns[indice_limon_24h:indice_teg_24h],inplace=True)


In [49]:
indice_limon_2gg = dati.columns.get_loc('limon_foll_2gg')
indice_teg_2gg = dati.columns.get_loc('ml_intem_foll_2gg')+1
dati.drop(columns=dati.columns[indice_limon_2gg:indice_teg_2gg],inplace=True)


In [50]:
indice_limon_3gg = dati.columns.get_loc('limon_foll_3gg')
indice_teg_3gg = dati.columns.get_loc('teg_foll_3gg')+1
dati.drop(columns=dati.columns[indice_limon_3gg:indice_teg_3gg],inplace=True)


In [51]:
dati.drop(columns=['colturali___1',
 'colturali___2',
 'colturali___3',
 'colturali___4',
 'colturali___5',
 'colturali___6',
 'colturali___7',
 'colturali___8',
 'colturali___10',
 'colturali___11',
 'colturali___9',
 'colturali_altro',
 'colt_pre_atb'],inplace=True)

In [52]:
dati.drop(columns=['colturali_foll_12h___1', 'colturali_foll_12h___2',
       'colturali_foll_12h___3', 'colturali_foll_12h___4',
       'colturali_foll_12h___5', 'colturali_foll_12h___6',
       'colturali_foll_12h___7', 'colturali_foll_12h___8',
       'colturali_foll_12h___10', 'colturali_foll_12h___11',
       'colturali_foll_12h___9', 'colturali_altro_foll_12h',
       'colt_pre_atb_foll_12h'],inplace=True)

In [53]:
dati.drop(columns=['colturali_foll_24h___1', 'colturali_foll_24h___2',
       'colturali_foll_24h___3', 'colturali_foll_24h___4',
       'colturali_foll_24h___5', 'colturali_foll_24h___6',
       'colturali_foll_24h___7', 'colturali_foll_24h___8',
       'colturali_foll_24h___10', 'colturali_foll_24h___11',
       'colturali_foll_24h___9', 'colturali_altro_foll_24h',
       'colt_pre_atb_foll_24h'],inplace=True)

In [54]:
dati.drop(columns=['colturali_foll_2gg___1', 'colturali_foll_2gg___2',
       'colturali_foll_2gg___3', 'colturali_foll_2gg___4',
       'colturali_foll_2gg___5', 'colturali_foll_2gg___6',
       'colturali_foll_2gg___7', 'colturali_foll_2gg___8',
       'colturali_foll_2gg___10', 'colturali_foll_2gg___11',
       'colturali_foll_2gg___9', 'colturali_altro_foll_2gg',
       'colt_pre_atb_foll_2gg'],inplace=True)

In [55]:
dati.drop(columns=['colturali_foll_3gg___1', 'colturali_foll_3gg___2',
       'colturali_foll_3gg___3', 'colturali_foll_3gg___4',
       'colturali_foll_3gg___5', 'colturali_foll_3gg___6',
       'colturali_foll_3gg___7', 'colturali_foll_3gg___8',
       'colturali_foll_3gg___10', 'colturali_foll_3gg___11',
       'colturali_foll_3gg___9', 'colturali_altro_foll_3gg',
       'colt_pre_atb_foll_3gg'],inplace=True)

In [56]:
dati['pos_coronavirus']=dati['pos_coronavirus'].fillna(3.0)
dati['pos_coronavirus_12h']=dati['pos_coronavirus_12h'].fillna(3.0)
dati['pos_coronavirus_24h']=dati['pos_coronavirus_24h'].fillna(3.0)
dati['pos_coronavirus_2gg']=dati['pos_coronavirus_2gg'].fillna(3.0)
dati['pos_coronavirus_3gg']=dati['pos_coronavirus_3gg'].fillna(3.0)

In [57]:
iniz_elim_covid = dati.columns.get_loc('pos_coronavirus')+1
fine__elim_covid = dati.columns.get_loc('pct')
dati.drop(columns=dati.columns[iniz_elim_covid:fine__elim_covid], inplace=True)

In [58]:
iniz_elim_covid_12h = dati.columns.get_loc('pos_coronavirus_12h')+1
fine__elim_covid_12h = dati.columns.get_loc('pct_foll_12h')
dati.drop(columns=dati.columns[iniz_elim_covid_12h:fine__elim_covid_12h], inplace=True)

In [59]:
iniz_elim_covid_24h = dati.columns.get_loc('pos_coronavirus_24h')+1
fine__elim_covid_24h = dati.columns.get_loc('pct_foll_24h')
dati.drop(columns=dati.columns[iniz_elim_covid_24h:fine__elim_covid_24h], inplace=True)

In [60]:
iniz_elim_covid_2gg = dati.columns.get_loc('pos_coronavirus_2gg')+1
fine__elim_covid_2gg = dati.columns.get_loc('pct_foll_2gg')
dati.drop(columns=dati.columns[iniz_elim_covid_2gg:fine__elim_covid_2gg], inplace=True)

In [61]:
iniz_elim_covid_3gg = dati.columns.get_loc('pos_coronavirus_3gg')+1
fine__elim_covid_3gg = dati.columns.get_loc('pct_foll_3gg')
dati.drop(columns=dati.columns[iniz_elim_covid_3gg:fine__elim_covid_3gg], inplace=True)

In [62]:
dati['wbc'] = dati['wbc']*(10**3)
dati['wbc_foll_12h']=dati['wbc_foll_12h'] * (10**3)
dati['wbc_foll_24h']=dati['wbc_foll_24h'] * (10**3)
dati['wbc_foll_2gg']=dati['wbc_foll_2gg'] * (10**3)
dati['wbc_foll_3gg']=dati['wbc_foll_3gg'] * (10**3)

In [63]:
index_cito = dati.columns.get_loc('cit')
index_flui= dati.columns.get_loc('fluidotp') + 1 

dati.drop(columns=dati.columns[index_cito:index_flui], inplace=True)

In [64]:
index_cito_12h = dati.columns.get_loc('cit_foll_12h')
index_flui_12h= dati.columns.get_loc('fluidotp_foll_12h') + 1 
dati.drop(columns=dati.columns[index_cito_12h:index_flui_12h], inplace=True)


In [65]:
index_cito_24h = dati.columns.get_loc('cit_foll_24h')
index_flui_24h= dati.columns.get_loc('fluidotp_foll_24h') + 1 
dati.drop(columns=dati.columns[index_cito_24h:index_flui_24h], inplace=True)


In [66]:
index_cito_2gg = dati.columns.get_loc('cit_foll_2gg')
index_flui_2gg= dati.columns.get_loc('fluidotp_foll_2gg') + 1 
dati.drop(columns=dati.columns[index_cito_2gg:index_flui_2gg], inplace=True)


In [67]:
index_cito_3gg = dati.columns.get_loc('cit_foll_3gg')
index_flui_3gg= dati.columns.get_loc('fluidotp_foll_3gg') + 1 
dati.drop(columns=dati.columns[index_cito_3gg:index_flui_3gg], inplace=True)


In [68]:
start_rimo = dati.columns.get_loc('altre_ter_extrac')
stop_rimo = dati.columns.get_loc('tempo_0_complete')+1
dati.drop(columns=dati.columns[start_rimo:stop_rimo], inplace=True)

In [69]:
start_rimo_12h = dati.columns.get_loc('altre_ter_extrac_12h')
stop_rimo_12h = dati.columns.get_loc('monitoraggio_12_ore_complete')+1

dati.drop(columns=dati.columns[start_rimo_12h:stop_rimo_12h], inplace=True)

In [70]:
start_rimo_24h = dati.columns.get_loc('altre_ter_extrac_24h')
stop_rimo_24h = dati.columns.get_loc('monitoraggio_24_ore_complete')+1
dati.drop(columns=dati.columns[start_rimo_24h:stop_rimo_24h], inplace=True)

In [71]:
start_rimo_2gg = dati.columns.get_loc('altre_ter_extrac_2gg')
stop_rimo_2gg = dati.columns.get_loc('monitoraggio_2_giorno_complete')+1

dati.drop(columns=dati.columns[start_rimo_2gg:stop_rimo_2gg], inplace=True)

In [72]:
start_rimo_3gg = dati.columns.get_loc('altre_ter_extrac_3gg')
stop_rimo_3gg = dati.columns.get_loc('monitoraggio_3_giorno_complete')+1

dati.drop(columns=dati.columns[start_rimo_3gg:stop_rimo_3gg], inplace=True)

In [73]:
dati['hct_crrt'] = dati['hct_crrt'].fillna(30.0)

In [74]:
eliminazione = []
for col in dati.columns:
    if dati[col].dtypes == 'O':
        eliminazione.append(col)
    

In [75]:
del(eliminazione[0])

In [76]:
eliminazione

['redcap_data_access_group',
 'ricovero',
 'tip_paz_chrurgico_altro',
 'n_cart_clin',
 'cog_paz',
 'nome_paz',
 'born',
 'comorb_altro',
 'avv_vent',
 'resp_spont_altro',
 'diur_3',
 'sede_altro',
 'arrt_start',
 'altro_filtro_crrt',
 'altro_coagu_crrt',
 'tip_camp_al_tp',
 'q_sam',
 'q_camp_tp',
 'data_new_access_foll_12h',
 'rimoz_access_foll_12h',
 'altro_sosp_foll_12h',
 'sost_foll_12h_altro',
 'altro_coagu_crrt_foll_12h',
 'avv_vent_12h',
 'resp_spont_altro_foll_12h',
 'data_new_access_foll_24h',
 'altro_sosp_foll_24h',
 'sost_0_foll_24h_altro',
 'altro_macc_crrt_foll_24h',
 'altro_coagu_crrt_foll_24h',
 'avv_vent_24h',
 'resp_spont_altro_foll_24h',
 'sede_altro_foll_24h',
 'data_new_access_foll_2gg',
 'rimoz_access_foll_2gg',
 'altro_sosp_foll_2gg',
 'sost_foll_2gg_altro',
 'altro_macc_crrt_foll_2gg',
 'altro_coagu_crrt_foll_2gg',
 'durata_irrt_foll_2gg',
 'avv_vent_2gg',
 'resp_spont_altro_foll_2gg',
 'sede_altro_foll_2gg',
 'data_new_access_foll_3gg',
 'rimoz_access_foll_3gg',


In [77]:
dati.drop(columns=eliminazione,inplace=True)

In [78]:
dati['fet_crrt'] = dati['fet_crrt'].fillna(3000.0)
dati['fet_crrt_foll_12h'] = dati['fet_crrt_foll_12h'].fillna(3000.0)
dati['fet_crrt_foll_24h'] = dati['fet_crrt_foll_24h'].fillna(3000.0)
dati['fet_crrt_foll_2gg'] = dati['fet_crrt_foll_2gg'].fillna(3000.0)
dati['fet_crrt_foll_3gg'] = dati['fet_crrt_foll_3gg'].fillna(3000.0)



In [82]:
dati['calc_ione_crrt'] = dati['calc_ione_crrt'].fillna(dati['ca_ione_crrt_mg'])

dati['ca_ione_crrt_foll_12h'] = dati['ca_ione_crrt_foll_12h'].fillna(dati['ca_ione_crrt_foll_12h_mg'])
dati['cal_post_filtro_crrt_foll_12h'] = dati['cal_post_filtro_crrt_foll_12h'].fillna(dati['cal_post_filtro_crrt_foll_12h_mg'])

dati['ca_ione_crrt_foll_24h'] = dati['ca_ione_crrt_foll_24h'].fillna(dati['ca_ione_crrt_foll_24h_mg'])
dati['cal_post_filtro_crrt_foll_24h'] = dati['cal_post_filtro_crrt_foll_24h'].fillna(dati['cal_post_filtro_crrt_foll_24h_mg'])

dati['ca_ione_crrt_foll_2gg'] = dati['ca_ione_crrt_foll_2gg'].fillna(dati['ca_ione_crrt_foll_2gg_mg'])
dati['ca_ione_crrt_foll_2gg'] = dati['ca_ione_crrt_foll_2gg'].fillna(dati['ca_ione_crrt_foll_2gg_mg'])

dati['ca_ione_crrt_foll_3gg'] = dati['ca_ione_crrt_foll_3gg'].fillna(dati['ca_ione_crrt_foll_3gg_mg'])
dati['ca_ione_crrt_foll_3gg'] = dati['ca_ione_crrt_foll_3gg'].fillna(dati['ca_ione_crrt_foll_3gg_mg'])


In [90]:
dati['record_id'].loc[(dati['deg_tot_dec']<=2) | (dati['deg_tot_dim']<=2)]

37       33-1
85      34-51
119     81-12
122     81-15
125     109-4
132     117-6
135     180-2
151    180-28
168    181-14
169    181-15
186     296-8
253    376-55
255    376-60
273     558-8
Name: record_id, dtype: object

In [93]:
df = dati[['creatinine_sofa_foll_2gg','gcs_sofa_foll_2gg','platelets_sofa_foll_2gg','bili_sofa_foll_2gg',
     'hypotension_sofa_foll_12h','resp_sofa_foll_12h','sofa_score_foll_2gg']]

In [119]:
dati.drop(dati.loc[(dati['record_id']=='297-42') | (dati['record_id']=='297-67')].index,inplace=True)
dati.reset_index(inplace=True)
dati.drop(columns=('index'),inplace=True)

In [135]:
dati['record_id'].loc[dati['gcs_foll_3gg'].isnull()]

16       22-3
20       22-7
21       22-8
22       22-9
23      22-10
        ...  
254    376-62
256    376-67
258     450-6
271     558-8
274    558-12
Name: record_id, Length: 70, dtype: object

In [136]:
dati = dati.fillna(0)

In [137]:
dati

,record_id,tip_paz,tip_ricov_chirurg,tip_paz_chirurgico,obiet_ricov_ti___1,obiet_ricov_ti___2,obiet_ricov_ti___3,obiet_ricov_ti___4,obiet_ricov_ti___5,obiet_ricov_ti___6,...,downtime_perc,dose_totale,ultrafiltr_totale,rac_cam_out,tip_camp_out___1,tip_camp_out___2,tip_camp_out___3,tip_camp_out___4,q_camp_out,outcome_ti_complete
0,4,1,0.0,0.0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0,0,0,0,0.0,2
1,5,1,0.0,0.0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0,0,0,0,0.0,2
2,6,1,0.0,0.0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0,0,0,0,0.0,2
3,7,1,0.0,0.0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0,0,0,0,0.0,2
4,8,1,0.0,0.0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0,0,0,0,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,558-5,1,0.0,0.0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0,0,0,0,0.0,2
271,558-8,2,2.0,1.0,1,1,1,0,0,0,...,0.0,0.0,0.0,0.0,0,0,0,0,0.0,2
272,558-9,1,0.0,0.0,1,1,1,0,0,0,...,0.0,0.0,0.0,0.0,0,0,0,0,0.0,2
273,558-10,1,0.0,0.0,1,1,1,0,1,0,...,0.0,0.0,0.0,0.0,0,0,0,0,0.0,2


In [138]:
dati.to_excel('dati_puliti_finali.xlsx')